## Start Clipper

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../../clipper_admin/'))
import clipper_manager as cm
reload(cm)
user = ""
key = ""
host = "localhost"
clipper = cm.Clipper(host, user, key)

In [ ]:
clipper.start()

In [ ]:
app_name = "example_app"
model_name = "example_model_name"
default_output = "0"
input_type = "doubles"

clipper.register_application(
    app_name,
    model_name,
    input_type,
    default_output,
    slo_micros=20000)

## Test setup

In [ ]:
def no_packages_needed_predict(inputs):
    outputs = []
    for input_item in inputs:
        outputs.append(str(sum(input_item)))
    return outputs

In [ ]:
def runtime_error_predict(inputs):
    1/0
    print("Shouldn't ever get here")

In [ ]:
def nonexistent_package_predict(inputs):
    import packagedoesntexist
    print("Shouldn't ever get here")

In [ ]:
# Make sure you run `conda install fuel` from within your local conda environment
def need_to_install_package_predict(inputs):
    import fuel
    print("Should get here after installation")
    outputs = []
    for input_item in inputs:
        outputs.append(str(sum(input_item)))
    return outputs

In [ ]:
def need_to_install_package_has_runtime_error_predict(inputs):
    import fuel
    print("Should get here after installation")
    1/0
    print("Shouldn't ever get here")

In [ ]:
import json
import requests
def predict(x):
    uid = 0
    url = "http://%s:1337/%s/predict" % (host, app_name)
    req_json = json.dumps({'uid': uid, 'input': list(x)})
    headers = {'Content-type': 'application/json'}
    r = requests.post(url, headers=headers, data=req_json)
    return r.text

In [ ]:
model_version = 50
datapoint = [1.0, 2.0]
labels = ["label1", "label2"]

# Test cases

## Outside of conda env: deploy function that needs no additional packages

**Management library should print:**

"Anaconda environment was either not found or failed being exported"

"Your local environment details will not be supplied to and loaded in the container in which your model is deployed."

**After startup, logs should show:**

"No dependencies supplied. Attempting to run Python container."


**Model-container should work and be queryable**

In [ ]:
clipper.deploy_predict_function(
    model_name,
    model_version,
    no_packages_needed_predict,
    labels,
    input_type,
)
model_version += 1

In [ ]:
predict(datapoint)

## Outside of conda env: deploy function that needs an additional package not pre-installed on the container

**Management library should print:**

"Anaconda environment was either not found or failed being exported"

"Your local environment details will not be supplied to and loaded in the container in which your model is deployed."

**After startup, logs should show:**

"No dependencies supplied. Attempting to run Python container."

**Upon query, logs should show:**

"Encountered an ImportError when running Python container."

"Please supply necessary dependencies through an Anaconda environment and try again."

In [ ]:
clipper.deploy_predict_function(
    model_name,
    model_version,
    nonexistent_package_predict, # This should have the same behavior with `need_to_install_package_predict` or `need_to_install_package_has_runtime_error_predict`
    labels,
    input_type,
)
model_version += 1

In [ ]:
predict(datapoint)

## Outside of conda env: deploy a function that experiences a runtime error

**Management library should print:**

"Anaconda environment was either not found or failed being exported"

"Your local environment details will not be supplied to and loaded in the container in which your model is deployed."

**After startup, logs should show:**

"No dependencies supplied. Attempting to run Python container."

**Upon query, logs should show:**

{Runtime error output, including "ZeroDivisionError: integer division or modulo by zero"}

"Encountered error not related to missing packages. Please refer to the above logs to diagnose."

In [ ]:
clipper.deploy_predict_function(
    model_name,
    model_version,
    runtime_error_predict,
    labels,
    input_type,
)
model_version += 1

In [ ]:
predict(datapoint)

## Within conda env: deploy a function needs no additional dependencies. Optimistic startup should work

**Management library should print:**

"Anaconda environment found. Verifying packages."

"Supplied environment details"

**After startup, logs should show:**

"Attempting to run Python container without installing without supplied dependencies."

**Model-container should work and be queryable**

In [ ]:
clipper.deploy_predict_function(
    model_name,
    model_version,
    no_packages_needed_predict,
    labels,
    input_type,
)
model_version += 1

In [ ]:
predict(datapoint)

## Within conda env: Deploy a function that will experience a runtime error

**Management library should print:**

"Anaconda environment found. Verifying packages."

"Supplied environment details"

**After startup, logs should show:**

"Attempting to run Python container without installing without supplied dependencies."

**Upon query, logs should show:**

"Encountered error not related to missing packages. Please refer to the above logs to diagnose."

In [ ]:
clipper.deploy_predict_function(
    model_name,
    model_version,
    runtime_error_predict,
    labels,
    input_type,
)
model_version += 1

In [ ]:
predict(datapoint)

## Within conda env: Deploy a function that uses a module not in the conda or pip distributions.

**Management library should print:**

"Anaconda environment found. Verifying packages."

"Supplied environment details"

**After startup, logs should show:**

"Attempting to run Python container without installing without supplied dependencies."

**Upon query, logs should show:**

"Encountered an ImportError when running Python container without installing supplied dependencies."

"Will install supplied dependencies and try again."

**Upon query after the supplied dependencies has been installed, logs should show:**

"Encountered an ImportError even after installing supplied dependencies."

In [ ]:
clipper.deploy_predict_function(
    model_name,
    model_version,
    nonexistent_package_predict,
    labels,
    input_type,
)
model_version += 1

In [ ]:
predict(datapoint)

## Within conda env: Deploy a function that uses a module not pre-installed but within the conda distribution

**Management library should print:**

"Anaconda environment found. Verifying packages."

"Supplied environment details"

**After startup, logs should show:**

"Attempting to run Python container without installing without supplied dependencies."

**Upon query, logs should show:**

"Encountered an ImportError when running Python container without installing supplied dependencies."

"Will install supplied dependencies and try again."

**Upon query after the supplied dependencies has been installed, model-container should be queryable and functional. Logs should show upon each query:**

"Should get here after installation" (from a print statement within the function)

In [ ]:
clipper.deploy_predict_function(
    model_name,
    model_version,
    need_to_install_package_predict,
    labels,
    input_type,
)
model_version += 1

In [ ]:
predict(datapoint)

## Within conda env: Deploy a function that uses a module not pre-installed but within the conda distribution that also will experience a runtime error

**Management library should print:**

"Anaconda environment found. Verifying packages."

"Supplied environment details"

**After startup, logs should show:**

"Attempting to run Python container without installing without supplied dependencies."

**Upon query, logs should show:**

"Encountered an ImportError when running Python container without installing supplied dependencies."

"Will install supplied dependencies and try again."

**Upon query after the supplied dependencies has been installed, logs should show:**

"Should get here after installation" (from a print statement within the function)

"Encountered error not related to missing packages. Please refer to the above logs to diagnose."

In [ ]:
clipper.deploy_predict_function(
    model_name,
    model_version,
    need_to_install_package_has_runtime_error_predict,
    labels,
    input_type,
)
model_version += 1

In [ ]:
predict(datapoint)